In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import time as time
import pyarrow as pa

In [2]:
import types
def imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            yield val.__name__
list(imports())

['builtins',
 'builtins',
 'matplotlib.pyplot',
 'numpy',
 'pandas',
 'xarray',
 'time',
 'pyarrow',
 'types']

In [39]:
def v_wind_eval(orig_height, target_height, wind_speeds, surface_roughness):
    """Evaluates the wind speed at a certain height using formula, outputs array"""
    
    alpha = 0.5*(surface_roughness/10)**0.2
    speeds = wind_speeds*(target_height/orig_height)**alpha
    
    return speed_arr




def wind_power_output(pwr_crv_str, speed_arr):
    """Uses power curve data from NRES and array of speeds to create correlated array of power outputs in MW of 
    that wind turbine in relation to each of those speeds. Power curve array has 3 columns: speed, power output, 
    capacity factor"""

    power_curve = np.loadtxt(pwr_crv_str,delimiter=",", dtype=str)
    power_curve = power_curve[1:,:].astype('float')
    speed_limited = np.where((speed_arr>25) | (speed_arr<0),0,speed_arr) #curtails power output above max
    power_outputs = np.interp(speed_limited,power_curve[:,0],power_curve[:,1])
    
    return power_outputs

In [41]:
import xarray as xr

# Open the GRIB file using xarray
x = xr.open_dataset('test.grib', engine='cfgrib')

# Print the dataset information
print(ds)

# Access specific variables or slices
u100_data = x['u100'].values
v100_data = x['v100'].values
latitude_values = x['latitude'].values
longitude_values = x['longitude'].values
time_value = x["time"].values


# Close the dataset
x.close()

x

<xarray.Dataset>
Dimensions:    (lat: 81, lon: 79, time: 1464)
Coordinates:
  * time       (time) datetime64[ns] 2022-11-01 ... 2022-12-31T23:00:00
Dimensions without coordinates: lat, lon
Data variables:
    latitude   (lat) float64 70.0 69.75 69.5 69.25 ... 50.75 50.5 50.25 50.0
    longitude  (lon) float64 -12.5 -12.25 -12.0 -11.75 ... 6.25 6.5 6.75 7.0
    u100       (time, lat, lon) float32 -10.24 -10.33 -10.41 ... 6.867 7.036
    v100       (time, lat, lon) float32 6.53 6.859 7.177 ... 10.34 10.26 9.914


<xarray.Dataset>
Dimensions:     (time: 1464, latitude: 81, longitude: 79)
Coordinates:
    number      int64 ...
  * time        (time) datetime64[ns] 2022-11-01 ... 2022-12-31T23:00:00
    step        timedelta64[ns] ...
    surface     float64 ...
  * latitude    (latitude) float64 70.0 69.75 69.5 69.25 ... 50.5 50.25 50.0
  * longitude   (longitude) float64 -12.5 -12.25 -12.0 -11.75 ... 6.5 6.75 7.0
    valid_time  (time) datetime64[ns] ...
Data variables:
    u100        (time, latitude, longitude) float32 -10.24 -10.33 ... 7.036
    v100        (time, latitude, longitude) float32 6.53 6.859 ... 10.26 9.914
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2023-11-14T15:04 GRIB to CDM+CF via cfgrib-0.9.1...

In [42]:
ds = xr.Dataset(
    {'latitude': (['lat'], latitude_values),
     'longitude': (['lon'], longitude_values),
     'u100': (['time', 'lat', 'lon'], u100_data),
     'v100': (['time', 'lat', 'lon'], v100_data)},
    coords={'time': ('time', time_value)}
)

ds

<xarray.Dataset>
Dimensions:    (lat: 81, lon: 79, time: 1464)
Coordinates:
  * time       (time) datetime64[ns] 2022-11-01 ... 2022-12-31T23:00:00
Dimensions without coordinates: lat, lon
Data variables:
    latitude   (lat) float64 70.0 69.75 69.5 69.25 ... 50.75 50.5 50.25 50.0
    longitude  (lon) float64 -12.5 -12.25 -12.0 -11.75 ... 6.25 6.5 6.75 7.0
    u100       (time, lat, lon) float32 -10.24 -10.33 -10.41 ... 6.867 7.036
    v100       (time, lat, lon) float32 6.53 6.859 7.177 ... 10.34 10.26 9.914

In [47]:
df = ds.to_dataframe()
df

latitude  longitude       u100       v100
lat lon time                                                          
0   0   2022-11-01 00:00:00      70.0      -12.5 -10.238571   6.529922
        2022-11-01 01:00:00      70.0      -12.5 -10.611816   5.772247
        2022-11-01 02:00:00      70.0      -12.5 -10.997757   5.493408
        2022-11-01 03:00:00      70.0      -12.5 -10.289139   6.455765
        2022-11-01 04:00:00      70.0      -12.5  -8.899551   7.919769
...                               ...        ...        ...        ...
80  78  2022-12-31 19:00:00      50.0        7.0   7.787338  10.291367
        2022-12-31 20:00:00      50.0        7.0   8.422897  10.266159
        2022-12-31 21:00:00      50.0        7.0   8.841629  10.174835
        2022-12-31 22:00:00      50.0        7.0   7.260818   9.767670
        2022-12-31 23:00:00      50.0        7.0   7.036148   9.913757

[9368136 rows x 4 columns]

KeyError: 0